In [1]:
import numpy as np
import pandas as pd
import datetime as dt
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Importing raw data with added external data
data = pd.read_csv('Hackathon Data Compressed.zip', compression= 'gzip')

In [3]:
data.head()

Encoded_SKU_ID  SALES_DATE      SUBCLASS_NAME           CLASS_NAME  \
0               1  2017-05-28  MISC CAMERA ACCES  DIGITAL CAMERA ACCY   
1               1  2017-05-29  MISC CAMERA ACCES  DIGITAL CAMERA ACCY   
2               1  2017-05-30  MISC CAMERA ACCES  DIGITAL CAMERA ACCY   
3               1  2017-05-31  MISC CAMERA ACCES  DIGITAL CAMERA ACCY   
4               1  2017-06-01  MISC CAMERA ACCES  DIGITAL CAMERA ACCY   

               ML_NAME         CATEGORY_NAME  RETAIL_PRICE  PROMO_PRICE  \
0  ML - DI Accessories  VP - Digital Imaging         89.99          NaN   
1  ML - DI Accessories  VP - Digital Imaging         89.99          NaN   
2  ML - DI Accessories  VP - Digital Imaging         89.99          NaN   
3  ML - DI Accessories  VP - Digital Imaging         89.99          NaN   
4  ML - DI Accessories  VP - Digital Imaging         89.99          NaN   

   COMPETITOR_PRICE      Inventory  DAILY_UNITS  month  year        Date  \
0               NaN  Fully-Stocked            0      5  2017  2017-05-31   
1               NaN  Fully-Stocked            0      5  2017  2017-05-31   
2               NaN  Fully-Stocked            0      5  2017  2017-05-31   
3               NaN  Fully-Stocked            0      5  2017  2017-05-31   
4               NaN  Fully-Stocked            0      6  2017  2017-06-30   

      gscpi  Month  Customer_Sentiment_Index  
0 -0.093900  April                      97.0  
1 -0.093900  April                      97.0  
2 -0.093900  April                      97.0  
3 -0.093900  April                      97.0  
4  0.127618   June                      95.0

In [4]:
data.columns

Index(['Encoded_SKU_ID', 'SALES_DATE', 'SUBCLASS_NAME', 'CLASS_NAME',
       'ML_NAME', 'CATEGORY_NAME', 'RETAIL_PRICE', 'PROMO_PRICE',
       'COMPETITOR_PRICE', 'Inventory', 'DAILY_UNITS', 'month', 'year', 'Date',
       'gscpi', 'Month', 'Customer_Sentiment_Index'],
      dtype='object')

In [5]:
data['SALES_DATE'] = pd.to_datetime(data['SALES_DATE'])
data = data.drop(columns= ['SUBCLASS_NAME', 'CLASS_NAME','ML_NAME', 'CATEGORY_NAME', 'RETAIL_PRICE', 'PROMO_PRICE','COMPETITOR_PRICE', 'Inventory', 'month', 'year', 'Date',
       'gscpi', 'Month', 'Customer_Sentiment_Index'])

In [6]:
data.head()

Encoded_SKU_ID SALES_DATE  DAILY_UNITS
0               1 2017-05-28            0
1               1 2017-05-29            0
2               1 2017-05-30            0
3               1 2017-05-31            0
4               1 2017-06-01            0

In [7]:
sku_list = list(data['Encoded_SKU_ID'].unique())
full_data = pd.DataFrame()
for sku in sku_list:
    # print(sku)
    single_sku = data[data['Encoded_SKU_ID'] == sku]
    min_date = single_sku.SALES_DATE.min()
    max_date = single_sku.SALES_DATE.max()
    single_sku_padded = pd.DataFrame({"date":pd.date_range(start=min_date,end=max_date)}).merge(single_sku,left_on="date",right_on="SALES_DATE",how="left")
    single_sku_padded['DAILY_UNITS'] =  single_sku_padded['DAILY_UNITS'].fillna(0)
    single_sku_padded['Encoded_SKU_ID'] = single_sku_padded['Encoded_SKU_ID'].fillna(sku) 
    full_data = pd.concat([full_data,single_sku_padded])




In [8]:
full_data.head()

date  Encoded_SKU_ID SALES_DATE  DAILY_UNITS
0 2017-05-28             1.0 2017-05-28          0.0
1 2017-05-29             1.0 2017-05-29          0.0
2 2017-05-30             1.0 2017-05-30          0.0
3 2017-05-31             1.0 2017-05-31          0.0
4 2017-06-01             1.0 2017-06-01          0.0

In [9]:
#imputing negative sales as 0 
full_data.loc[full_data['DAILY_UNITS'] < 0 ,'DAILY_UNITS'] = 0 

In [697]:
# full_data.head()

In [10]:
full_data = full_data.drop(columns= ['SALES_DATE'])

In [11]:
full_data['date'] = pd.to_datetime(full_data['date'])

In [700]:
# avg_pred = full_data.groupby('Encoded_SKU_ID', as_index= False)['DAILY_UNITS'].mean()

In [12]:
full_data.head()

date  Encoded_SKU_ID  DAILY_UNITS
0 2017-05-28             1.0          0.0
1 2017-05-29             1.0          0.0
2 2017-05-30             1.0          0.0
3 2017-05-31             1.0          0.0
4 2017-06-01             1.0          0.0

In [14]:
#Create the Croston model function


#Croston's Method - https://medium.com/analytics-vidhya/croston-forecast-model-for-intermittent-demand-360287a17f5f
def Croston_TSB(ts,extra_periods=1,alpha=0.83,beta=0.01):
    d = np.array(ts) # Transform the input into a numpy array
    cols = len(d) # Historical period length
    d = np.append(d,[np.nan]*extra_periods) # Append np.nan into the demand array to cover future periods
    
    #level (a), probability(p) and forecast (f)
    a,p,f = np.full((3,cols+extra_periods),np.nan)
    # print(a)
    # print(p)
    # print(f)
    
# Initialization
    first_occurence = np.argmax(d[:cols]>0)
    a[0] = d[first_occurence]
    p[0] = 1/(1 + first_occurence)
    f[0] = p[0]*a[0]
                 
    # Create all the t+1 forecasts
    for t in range(0,cols): 
        if d[t] > 0:
            a[t+1] = alpha*d[t] + (1-alpha)*a[t] 
            p[t+1] = beta*(1) + (1-beta)*p[t]  
        else:
            a[t+1] = a[t]
            p[t+1] = (1-beta)*p[t]       
        f[t+1] = p[t+1]*a[t+1]
        
    # Future Forecast
    a[cols+1:cols+extra_periods] = a[cols]
    p[cols+1:cols+extra_periods] = p[cols]
    f[cols+1:cols+extra_periods] = f[cols]
    print(a.shape)
    print(p.shape)
    print(f.shape) 
    print(d.shape)                 
    df = pd.DataFrame.from_dict({"Demand":d,"Forecast":f,"Period":p,"Level":a,"Error":d-f})
    return df

In [703]:
# sku_list = list(full_data['Encoded_SKU_ID'].unique())
# sku_list.index(485)

In [16]:
croston_series_all = []
sku_list = list(full_data['Encoded_SKU_ID'].unique())
for i,sku in enumerate(sku_list):
    # print(i,sku)
    df = full_data.loc[full_data['Encoded_SKU_ID'] == sku]
    df.drop(columns= 'Encoded_SKU_ID', inplace= True)
    df.set_index(df['date'], inplace=True, drop=True)
    df.drop(columns= 'date', inplace= True)
    res= Croston_TSB(df,extra_periods=7)
    croston_series_all.append(res)

(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1891,)
(1891,)
(1891,)
(1891,)
(1590,)
(1590,)
(1590,)
(1590,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1044,)
(1044,)
(1044,)
(1044,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(735,)
(735,)
(735,)
(735,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(736,)
(736,)
(736,)
(736,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1898,)
(1106,)
(1106,)
(1106,)
(1106,)
(1898,)
(1898,)
(1898,)
(1898,)
(1590,)
(1590,)
(1590,)
(1590,)
(1884,)
(1884,)
(1884,)
(1884,)
(1898,)
(1898,)
(1898,)
(1898,)
(1891,)
(1891,)
(1891,)
(1891,)
(1898,)
(1898,)
(1898,)
(1898,)
(694,)
(694,)
(694,)
(694,)
(1590,)
(1590,)
(1590,)
(1590,)
(1898,)
(1898,)
(1898,)
(1898,)
(1590,)
(1590,)
(1590,)
(1590,)
(993,)
(993,)
(993,)
(993,)
(1898,)
(1898,)
(1898,)


In [18]:
#example of an output
croston_series_all[1]

Demand  Forecast    Period      Level     Error
0        0.0  5.000000  0.166667  30.000000 -5.000000
1        0.0  4.950000  0.165000  30.000000 -4.950000
2        0.0  4.900500  0.163350  30.000000 -4.900500
3        0.0  4.851495  0.161716  30.000000 -4.851495
4        0.0  4.802980  0.160099  30.000000 -4.802980
...      ...       ...       ...        ...       ...
1893     NaN  2.094939  0.961281   2.179319       NaN
1894     NaN  2.094939  0.961281   2.179319       NaN
1895     NaN  2.094939  0.961281   2.179319       NaN
1896     NaN  2.094939  0.961281   2.179319       NaN
1897     NaN  2.094939  0.961281   2.179319       NaN

[1898 rows x 5 columns]

In [19]:
#Adding all forecasts together
final_df = pd.DataFrame(columns=['pred','sku','dt'])
date_range = pd.date_range(start='8/1/2022', end='8/07/2022')
for i in range(len(sku_list)):
    prediction = list(croston_series_all[i]['Forecast'].tail(7))
    sku = [sku_list[i]]*7
    df = pd.DataFrame(columns=['pred','sku','dt'])
    df['sku'] = sku
    df['pred'] =  prediction
    df['dt'] = date_range
    final_df = pd.concat([final_df,df])


In [20]:
final_df

pred    sku         dt
0   0.807129    1.0 2022-08-01
1   0.807129    1.0 2022-08-02
2   0.807129    1.0 2022-08-03
3   0.807129    1.0 2022-08-04
4   0.807129    1.0 2022-08-05
..       ...    ...        ...
2   0.671712  575.0 2022-08-03
3   0.671712  575.0 2022-08-04
4   0.671712  575.0 2022-08-05
5   0.671712  575.0 2022-08-06
6   0.671712  575.0 2022-08-07

[4025 rows x 3 columns]

In [21]:
validation_data = pd.read_excel('Validation_Data.xlsx')

In [22]:
validation_data_clean = validation_data[['Encoded_SKU_ID', 'SALES_DATE', 'DAILY_UNITS']]
validation_data_clean['SALES_DATE'] = pd.to_datetime(validation_data_clean['SALES_DATE'])

In [24]:
validation_data_clean = validation_data_clean.sort_values(by = ['Encoded_SKU_ID', 'SALES_DATE'])
# data_forecast_pred = validation_data_clean.merge(final_df, left_on = ['Encoded_SKU_ID', 'SALES_DATE'], right_on = ['sku', 'dt'])
data_forecast_pred = validation_data_clean.merge(final_df, left_on = ['Encoded_SKU_ID', 'SALES_DATE'], right_on = ['sku', 'dt'])

In [25]:
data_forecast_pred

Encoded_SKU_ID SALES_DATE  DAILY_UNITS      pred    sku         dt
0                  1 2022-08-01            0  0.807129    1.0 2022-08-01
1                  1 2022-08-02            1  0.807129    1.0 2022-08-02
2                  1 2022-08-03            0  0.807129    1.0 2022-08-03
3                  1 2022-08-04            0  0.807129    1.0 2022-08-04
4                  1 2022-08-05            1  0.807129    1.0 2022-08-05
...              ...        ...          ...       ...    ...        ...
3768             575 2022-08-03            1  0.671712  575.0 2022-08-03
3769             575 2022-08-04            0  0.671712  575.0 2022-08-04
3770             575 2022-08-05            0  0.671712  575.0 2022-08-05
3771             575 2022-08-06            0  0.671712  575.0 2022-08-06
3772             575 2022-08-07            0  0.671712  575.0 2022-08-07

[3773 rows x 6 columns]

In [26]:
#test the next seven days
from sklearn import metrics
mae = metrics.mean_absolute_error(round(data_forecast_pred['pred'],0), data_forecast_pred['DAILY_UNITS'])
mse = metrics.mean_squared_error(round(data_forecast_pred['pred'],0), data_forecast_pred['DAILY_UNITS'])
rmse = np.sqrt(mse)  
r2 = metrics.r2_score(round(data_forecast_pred['pred'],0), data_forecast_pred['DAILY_UNITS'])

print("Results of Croston")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

Results of Croston
MAE: 1.7580174927113703
MSE: 22.253644314868804
RMSE: 4.717376846815273
R-Squared: 0.9342791740436105
